In [2]:
file_path = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_09_Angus_River_waters.csv'
file_path = '/Users/angus/work/sukinda/lab_work/cations/raw_data/2025_01_13_ANGUS_RIVER_WATERS.csv'
%run main.py $file_path
%run unit_conv.py 'output.nc'

from utils import select_vars_by_element as sel_element
from utils import select_by_fuzzy_sample as search_samples
from utils import find_common_elements as common_elements
from utils import find_missing_elements as missing_elements
from utils import average_wavelengths
from check_stds import check_standards
chk_stds = check_standards()

<xarray.Dataset>
Dimensions:                      (sample_name: 61, species: 39)
Coordinates:
  * sample_name                  (sample_name) object 'A' 'BW' 'B' ... 'T8' 'T9'
  * species                      (species) object 'Al 396.152 Ax' ... 'Sr 421...
Data variables: (12/29)
    Flags                        (sample_name, species) object '' '' ... '' ''
    Unadjusted_Data              (sample_name, species) float64 0.0027 ... 0....
    Concentration                (sample_name, species) float64 0.0027 ... 0....
    Intensity                    (sample_name, species) float64 108.0 ... 5.0...
    Intensity_SD                 (sample_name, species) float64 6.237 ... 105.1
    Intensity___RSD              (sample_name, species) float64 5.77 ... 0.21
    ...                           ...
    Flags_Replicate_1            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_2            (sample_name, species) object '' '' ... '' ''
    Flags_Replicate_3            (sample_nam

/Users/angus/work/icp_oes/main.py:188: UserWarning: rename 'sample_name' to 'sample_name' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)
/Users/angus/work/icp_oes/main.py:188: UserWarning: rename 'species' to 'species' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename(coord_mapping)


In [3]:
good_wavelengths = {}
possible_wavelengths = {}
for standard in chk_stds:
    print(f'*** {standard} ***')
    good_wavelengths[standard] = []
    possible_wavelengths[standard] = []
    for element in ATOMIC_WEIGHTS.keys():
        print(f'*** {element} ***')
        try:
            certified = chk_stds[standard][0][element]
            uncertainty = chk_stds[standard][1][element]
            print(f'{standard} certified value: {certified:.3g} ± {uncertainty:.3g} mM')
        except KeyError:
            print(f'{standard} does not have a certified value for {element}')
            continue
        na_ds = sel_element(ds, element)
        na_ds = search_samples(na_ds, standard)
        for specie in na_ds.coords['species'].values:
            mean = na_ds.sel(species=specie)['Concentration_mM'].mean()
            two_sd = na_ds.sel(species=specie)['Concentration_mM'].std() * 2
            percentage_diff = (mean - certified) / certified * 100
            print(f'{specie}: {mean:.3g} ± {two_sd:.3g} mM, {percentage_diff:.2f}%')
            if abs(percentage_diff) < 5:
                good_wavelengths[standard].append(specie)
            elif abs(percentage_diff) > 5 and abs(percentage_diff) < 10:
                possible_wavelengths[standard].append(specie)

/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


*** SLRS-6 ***
*** Al ***
SLRS-6 certified value: 0.00125 ± 8.2e-05 mM
Al 396.152 Ax: 0.00133 ± 4.58e-05 mM, 6.03%
Al 396.152 R: 0.00128 ± 0.000223 mM, 2.47%
*** Ba ***
SLRS-6 certified value: 0.000104 ± 3.5e-06 mM
Ba 455.403 Ax: 0.000104 ± 1.98e-06 mM, -0.15%
Ba 455.403 R: 0.000104 ± 2.14e-06 mM, 0.27%
*** Ca ***
SLRS-6 certified value: 0.218 ± 0.005 mM
Ca 315.887 R: 0.223 ± 0.00384 mM, 2.08%
Ca 317.933 R: 0.222 ± 0.00351 mM, 1.95%
Ca 396.847 Ax: 0.206 ± 0.0037 mM, -5.49%
Ca 396.847 R: 0.21 ± 0.00215 mM, -3.80%
Ca 422.673 R: 0.211 ± 0.0035 mM, -3.11%
*** Fe ***
SLRS-6 certified value: 0.00151 ± 6.4e-05 mM
Fe 238.204 R: 0.00146 ± 3.71e-05 mM, -3.61%
Fe 259.940 Ax: 0.00144 ± 1.37e-05 mM, -4.44%
Fe 259.940 R: 0.00145 ± 9.45e-06 mM, -3.73%
*** K ***
SLRS-6 certified value: 0.0167 ± 0.0014 mM
K 766.491 Ax: 0.0169 ± 0.000296 mM, 1.07%
K 766.491 R: nan ± nan mM, nan%
K 769.897 Ax: 0.0172 ± 0.000276 mM, 2.74%
*** Mg ***
SLRS-6 certified value: 0.0877 ± 0.0024 mM
Mg 279.078 R: 0.0869 ± 0.00171

/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/angus/miniconda3/envs/topepan/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [4]:
good_list = common_elements(good_wavelengths)
good_list

{'Ba 455.403 Ax',
 'Ba 455.403 R',
 'Ca 315.887 R',
 'Ca 317.933 R',
 'Ca 396.847 R',
 'Ca 422.673 R',
 'Mg 279.078 R',
 'Mg 279.553 R',
 'Mg 279.800 R',
 'Mg 280.270 R',
 'Mg 285.213 Ax',
 'Mg 285.213 R',
 'Sr 407.771 Ax',
 'Sr 407.771 R',
 'Sr 421.552 Ax',
 'Sr 421.552 R'}

In [5]:
avg_da = average_wavelengths(ds, good_list)

In [6]:
avg_da.sel(sample_name='SLRS-6_2')

<xarray.DataArray 'Concentration_mM' (species: 4)>
array([4.51951609e-04, 1.04131016e-04, 2.16547732e-01, 8.63690599e-02])
Coordinates:
    sample_name  <U8 'SLRS-6_2'
  * species      (species) <U10 'Sr_average' 'Ba_average' ... 'Mg_average'

In [12]:
import hvplot.xarray  # noqa
import xarray as xr

violin_plot = avg_da.hvplot.violin(y='Concentration_mM', by='species')
special_point = avg_da.sel(sample_name='SLRS-6_2')  
point_plot = special_point.hvplot.scatter(x='species', y='Concentration_mM', color='red',
                                           marker='*', size=15)
combined_plot = violin_plot * point_plot
combined_plot


:Overlay
   .Violin.I  :Violin   [species]   (Concentration_mM)
   .Scatter.I :Scatter   [species]   (Concentration_mM)

In [ ]:
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')  # or your preferred backend

# Assuming your xarray dataset is named ds and has coordinates 'species' and 'sample_name'
species_list = ds.coords['species'].values

plots = []

for sp in species_list:
    # Filter the data for the current species
    species_data = ds.sel(species=sp)
    
    # Create a violin plot for this species.
    # Here, x is 'sample_name' (for example) and y is the variable of interest ('value').
    violin = species_data.hvplot.violin(
        x='sample_name',
        y='Concentration_mM',
        title=f"Species: {sp}",
        width=300,  # adjust as needed
        height=300
    )
    
    # Try to filter for the special sample in this species
    try:
        special_point = species_data.sel(sample_name='special_sample')
        # Create a scatter plot for the special point
        special_scatter = special_point.hvplot.scatter(
            x='sample_name',
            y='Concentration_mM',
            color='red',
            marker='*',
            size=15
        )
        # Overlay the special point on the violin plot
        combined = violin * special_scatter
    except KeyError:
        # If the special sample isn't present for this species, just use the violin plot
        combined = violin

    plots.append(combined)

# Arrange all species plots into a grid with, e.g., 3 columns.
layout = hv.Layout(plots).cols(3)
layout
